In [1]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, f1_score, jaccard_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import kagglehub
dataset_path = kagglehub.dataset_download("manideep1108/tusimple")
dataset_path = os.path.join(dataset_path, 'TUSimple')
train_set_path = os.path.join(dataset_path, 'train_set')
test_set_path = os.path.join(dataset_path, 'test_set')
test_json_path = os.path.join(test_set_path, 'test_tasks_0627.json')

def load_annotations(json_paths, image_base_path):
    annotations = []
    for json_path in json_paths:
        if not os.path.exists(json_path):
            print(f"[Warning] JSON file not found: {json_path}")
            continue
        with open(json_path, 'r') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    image_path = os.path.join(image_base_path, data['raw_file'])
                    if os.path.exists(image_path):
                        data['image_path'] = image_path
                        annotations.append(data)
                    else:
                        print(f"[Missing Image] {image_path}")
                except json.JSONDecodeError as e:
                    print(f"[JSON Error] in {json_path}: {e}")
    return annotations

train_json_files = ['label_data_0313.json', 'label_data_0531.json', 'label_data_0601.json']
train_json_paths = [os.path.join(train_set_path, f) for f in train_json_files]
train_annotations = load_annotations(train_json_paths, train_set_path)
test_annotations = load_annotations([test_json_path], test_set_path)
print(f"[Info] Train Annotations: {len(train_annotations)}, Test Annotations: {len(test_annotations)}")

class LaneDataset(Dataset):
    def __init__(self, annotations):
        self.annotations = annotations

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img = cv2.imread(ann['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 128)) / 255.0
        img = (img - img.mean()) / img.std() #  normalization using z
        img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1)

        mask = np.zeros((128, 256), dtype=np.uint8) #for semantic segmentation
        for lane in ann['lanes']: #draw lane points(white) on mask(black bckgrnd)
            for x, y in zip(lane, ann['h_samples']):
                if x != -2:
                    x = int(x * 256 / 1280)
                    y = int(y * 128 / 720)
                    if 0 <= x < 256 and 0 <= y < 128:
                        cv2.circle(mask, (x, y), 2, 1, -1)
        mask = torch.tensor(mask, dtype=torch.long)
        return img, mask

train_data, val_data = train_test_split(train_annotations, test_size=0.2, random_state=42)
train_loader = DataLoader(LaneDataset(train_data), batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(LaneDataset(val_data), batch_size=16, num_workers=2)

class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        def conv_block(in_c, out_c):
            return nn.Sequential(nn.Conv2d(in_c, out_c, 3, padding=1),
                                 nn.ReLU(),
                                 nn.Conv2d(out_c, out_c, 3, padding=1),
                                 nn.ReLU())
        self.down1 = conv_block(3, 32)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = conv_block(32, 64)
        self.pool2 = nn.MaxPool2d(2)
        self.bridge = conv_block(64, 128)
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = conv_block(128, 64)
        self.up1 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec1 = conv_block(64, 32)
        self.final = nn.Conv2d(32, 2, 1)

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(self.pool1(d1))
        b = self.bridge(self.pool2(d2))
        u2 = self.dec2(torch.cat([self.up2(b), d2], dim=1))
        u1 = self.dec1(torch.cat([self.up1(u2), d1], dim=1))
        return self.final(u1)

class FastSCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 2, kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class ENet(nn.Module):
    def __init__(self):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU()
        )
        self.bottleneck = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.final = nn.ConvTranspose2d(32, 2, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = self.initial(x)
        x = self.bottleneck(x)
        x = self.final(x)
        return x

def train_and_evaluate(model, name):
    model.to(device)
    weights = torch.tensor([0.1, 1.0], device=device)
    criterion = nn.CrossEntropyLoss(weight=weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_losses = []
    for epoch in range(10):
        model.train()
        total_loss = 0
        for imgs, masks in tqdm(train_loader):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = criterion(preds, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f"{name} - Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device)
            masks = masks.numpy()
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()
            y_true.extend(masks.flatten())
            y_pred.extend(preds.flatten())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    iou = jaccard_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)

    print(f"{name} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, F1: {f1:.4f}, IoU: {iou:.4f}")
    print(f"{name} - Confusion Matrix:\n{cm}")

    plt.plot(train_losses, label=name)
    plt.title(f"Training Loss - {name}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(f"{name}_loss.png")
    plt.clf()

    torch.save(model.state_dict(), f"{name}_lane_detection.pth")
    print(f"{name} model saved as '{name}_lane_detection.pth'")
    return model

models = {
    "UNet": UNet(),
    "FastSCNN": FastSCNN(),
    "ENet": ENet()
}

for name, model in models.items():
    trained_model = train_and_evaluate(model, name)
    trained_model.eval()
    imgs, masks = next(iter(val_loader))
    imgs = imgs.to(device)
    with torch.no_grad():
        preds = trained_model(imgs).argmax(dim=1).cpu().numpy()
    imgs = imgs.cpu().permute(0, 2, 3, 1).numpy()
    masks = masks.numpy()
    for i in range(min(2, len(imgs))):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.title("Image")
        plt.imshow(imgs[i])
        plt.axis('off')
        plt.subplot(1, 3, 2)
        plt.title("Ground Truth")
        plt.imshow(masks[i])
        plt.axis('off')
        plt.subplot(1, 3, 3)
        plt.title("Prediction")
        plt.imshow(preds[i])
        plt.axis('off')
        plt.savefig(f"{name}_visualization_{i}.png")
        plt.close()

[Info] Train Annotations: 3626, Test Annotations: 2782


100%|██████████| 182/182 [00:32<00:00,  5.58it/s]


UNet - Epoch 1, Loss: 0.4382


100%|██████████| 182/182 [00:29<00:00,  6.20it/s]


UNet - Epoch 2, Loss: 0.2729


...

ENet - Epoch 9, Loss: 0.3775


100%|██████████| 182/182 [00:18<00:00, 10.01it/s]

ENet - Epoch 10, Loss: 0.3748


ENet - Accuracy: 0.8881, Precision: 0.2133, F1: 0.3231, IoU: 0.1927
ENet - Confusion Matrix:
[[20492226  2343098]
 [  318826   635418]]
ENet model saved as 'ENet_lane_detection.pth'


<Figure size 640x480 with 0 Axes>